In [ ]:
import pandas as pd
from keras.models import Sequential  # type: ignore
from keras.layers import LSTM, Dense  # type: ignore
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
# Step 1: Load the dataset
data = pd.read_csv('crypto_yearly_data.csv')

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
# Function to plot a column grouped by year
def plot_grouped_by_year(column_name):
    plt.figure(figsize=(15, 5))
    data.groupby('year')[column_name].mean().plot()
    plt.xlabel("Year")
    plt.ylabel(column_name)
    plt.title(f"{column_name} by Year")
    plt.show()

In [ ]:
plot_grouped_by_year('open')
plot_grouped_by_year('close')
plot_grouped_by_year('low')
plot_grouped_by_year('high')
plot_grouped_by_year('volume')


In [ ]:
data = data.drop('year',axis=1)

In [ ]:
data.columns

In [ ]:
Adj_close_price = data[['close']]

In [ ]:
max(Adj_close_price.values),min(Adj_close_price.values) 

In [ ]:
def prepare_data(data):
    """
    Prepare data for LSTM model by creating sliding windows of features and labels.
    """
    
    # Scale the close prices
    scaler = MinMaxScaler(feature_range=(0, 1))
    data['close_scaled'] = scaler.fit_transform(data[['close']])
    
    # Create features and labels
    features, labels = [], []
    # Adjust loop to ensure valid indexing
    for i in range(len(data) - 100):
        features.append(data['close_scaled'].values[i:i + 100])
        labels.append(data['close_scaled'].values[i + 100])
    
    return np.array(features), np.array(labels), scaler

In [ ]:
def train_lstm_model(features, labels):
    """
    Define, train, and return an LSTM model.
    """
    # Reshape data for LSTM
    features = features.reshape((features.shape[0], features.shape[1], 1))
    
    # Split data into training and testing sets
    train_size = int(len(features) * 0.8)
    X_train, X_test = features[:train_size], features[train_size:]
    y_train, y_test = labels[:train_size], labels[train_size:]

    # Define LSTM model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mse') 
    
    # Train the model
    print("Training the LSTM model...")
    model.fit(X_train, y_train, epochs=2, batch_size=32, verbose=1)
    
    # Display model summary
    model.summary() 
    
    return model, X_test, y_test


In [ ]:
def evaluate_and_visualize(model, X_test, y_test, scaler):
    """
    Generate predictions, evaluate metrics, and visualize results.
    """
    predictions = model.predict(X_test)
    
    # Inverse scale predictions and actual values
    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Evaluate metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)

    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R² Score: {r2}")

In [ ]:
# Save the model
def save_model(model, file_path="complete_train.keras"):
    """
    Save the trained model to a file.
    """
    model.save(file_path)
    print(f"Model saved to {file_path}.")


In [ ]:
model_file = "complete_train.keras"

if not data.empty:
    # Prepare data for AI model
    features, labels, scaler = prepare_data(data)

    # Train LSTM model
    model, X_test, y_test = train_lstm_model(features, labels)
    

    # Save the model
    save_model(model, file_path=model_file)

    # Evaluate and visualize results
    evaluate_and_visualize(model, X_test, y_test, scaler)
else:
    print("Data not found or empty. Exiting...")
